# Voyages API Voyages Like Use Case

The purpose of this example is to produce an excel file similar to the output that a user of Signal Ocean platform would get if he/she downloaded the results of Voyages Data Dashboard https://www.signalocean.com/

## Setup
Install the Signal Ocean SDK:
```
pip install signal-ocean
```
Set your subscription key acquired here: https://apis.signalocean.com/profile

In [1]:
signal_ocean_api_key = '' #replace with your subscription key

In [3]:
from signal_ocean import Connection
from signal_ocean.voyages import VoyagesAPI
import pandas as pd
import numpy as np
from datetime import date, timedelta
import datetime as dt

In [4]:
connection = Connection(signal_ocean_api_key)
api = VoyagesAPI(connection)

### Get voyages

In [5]:
vlcc_id = 84
date_from = date.today() - timedelta(days=100)

For this example we will also need a relaxed_date_from variable in order to increase the chances of retrieving the attributes concerning the opening of the vessel. More specifically while someone may be interested in the voyages of the last X months for a specific vessel class, the opening info of the first voyage of each vessel are equivalent to the one of the last discharge of the previous voyage of the same vessel.For this reason we should be confident that the previous voyage is retrieved as well.

In [6]:
relaxed_date_from = date_from - timedelta(days=360)
voyages = api.get_voyages(vessel_class_id=vlcc_id, date_from=relaxed_date_from)

In [7]:
voyages = pd.DataFrame(v.__dict__ for v in voyages)
events = pd.DataFrame(e.__dict__ for voyage_events in voyages['events'].dropna() for e in voyage_events)

In [8]:
for col in voyages.columns.tolist():
    if str(voyages[col].dtypes) =='datetime64[ns, UTC]':
        voyages[col] = pd.to_datetime(voyages[col].astype(str).str[:-6])

In [9]:
voyages_of_interest = voyages[voyages['start_date'] >= pd.to_datetime(date_from)].copy()

In [10]:
voyages.reset_index(inplace=True)

In [11]:
voyage_ids = voyages.set_index(['imo', 'voyage_number'])['id'].to_dict()
previous_voyage_id = {voyage_id: voyage_ids[(imo, voyage_number-1)] 
                  for voyage_id, imo, voyage_number in voyages[['id', 'imo', 'voyage_number']].values 
                  if (imo, voyage_number-1) in voyage_ids}
voyages_of_interest['prev_id'] = voyages.id.map(previous_voyage_id)
voyages.set_index('id',inplace=True)

In [12]:
def get_opening_event(id_of_prev_voy):
    return next((e.__dict__ for e in reversed(voyages.loc[id_of_prev_voy].events or []) if e.purpose=='Discharge'),None)
    
voyages_of_interest.loc[voyages_of_interest.prev_id.notna(),'opening_event'] = \
voyages_of_interest.loc[voyages_of_interest.prev_id.notna(),'prev_id'].apply(get_opening_event)

voy_df_for_iterations = voyages_of_interest.dropna(axis=0, subset=['prev_id'])

voyages_of_interest.loc[voyages_of_interest.prev_id.notna(),'open_port'] = \
[ event['port_name'] for event in voy_df_for_iterations.opening_event]

voyages_of_interest.loc[voyages_of_interest.prev_id.notna(),'narrow_area'] = \
[ event['area_name_level0'] for event in voy_df_for_iterations.opening_event]

voyages_of_interest.loc[voyages_of_interest.prev_id.notna(),'wide_area'] = \
[ event['area_name_level2'] for event in voy_df_for_iterations.opening_event]

voyages_of_interest.loc[voyages_of_interest.prev_id.notna(),'area'] = \
[ event['area_name_level1'] for event in voy_df_for_iterations.opening_event]

voyages_of_interest.loc[voyages_of_interest.prev_id.notna(),'starting_port_sail_date'] = \
[ event['sailing_date'] for event in voy_df_for_iterations.opening_event]

voyages_of_interest.loc[voyages_of_interest.prev_id.notna(),'starting_port_longitude'] = \
[ event['longitude'] for event in voy_df_for_iterations.opening_event]

voyages_of_interest.loc[voyages_of_interest.prev_id.notna(),'starting_port_lantitude'] = \
[ event['latitude'] for event in voy_df_for_iterations.opening_event]

In [13]:
def get_load_event(voyage_events):
    return next((e.__dict__ for e in voyage_events or [] if e.purpose=='Load'), None)

voyages_of_interest['load_event'] = voyages_of_interest['events'].apply(get_load_event)
voy_df_for_iterations = voyages_of_interest.dropna(axis=0, subset=['load_event'])

voyages_of_interest.loc[voyages_of_interest.load_event.notna(),'load_port'] = \
[ event['port_name'] for event in voy_df_for_iterations.load_event]

voyages_of_interest.loc[voyages_of_interest.load_event.notna(),'load_country'] = \
[ event['country'] for event in voy_df_for_iterations.load_event]

voyages_of_interest.loc[voyages_of_interest.load_event.notna(),'load_port_arrival_date'] = \
[ event['arrival_date'] for event in voy_df_for_iterations.load_event]

voyages_of_interest.loc[voyages_of_interest.load_event.notna(),'load_port_sailing_date'] = \
[ event['sailing_date'] for event in voy_df_for_iterations.load_event]

voyages_of_interest.loc[voyages_of_interest.load_event.notna(),'load_port_narrow_area'] = \
[ event['area_name_level0'] for event in voy_df_for_iterations.load_event]

voyages_of_interest.loc[voyages_of_interest.load_event.notna(),'load_port_wider_area'] = \
[ event['area_name_level2'] for event in voy_df_for_iterations.load_event]

voyages_of_interest.loc[voyages_of_interest.load_event.notna(),'load_port_area'] = \
[ event['area_name_level1'] for event in voy_df_for_iterations.load_event]

voyages_of_interest.loc[voyages_of_interest.load_event.notna(),'load_port_longitude'] = \
[ event['longitude'] for event in voy_df_for_iterations.load_event]

voyages_of_interest.loc[voyages_of_interest.load_event.notna(),'load_port_latitude'] = \
[ event['latitude'] for event in voy_df_for_iterations.load_event]

voyages_of_interest.loc[voyages_of_interest.load_event.notna(),'predicted_l'] = \
[ event['event_horizon'] for event in voy_df_for_iterations.load_event]

In [14]:
def get_discharge_event(voyage_events):
    return next((e.__dict__ for e in reversed(voyage_events or []) if e.purpose=='Discharge'), None)

voyages_of_interest['discharge_event'] = voyages_of_interest['events'].apply(get_discharge_event)
voy_df_for_iterations = voyages_of_interest.dropna(axis=0, subset=['discharge_event'])

voyages_of_interest.loc[voyages_of_interest.discharge_event.notna(),'discharge_port'] = \
[ event['port_name'] for event in voy_df_for_iterations.discharge_event]

voyages_of_interest.loc[voyages_of_interest.discharge_event.notna(),'discharge_country'] = \
[ event['country'] for event in voy_df_for_iterations.discharge_event]

voyages_of_interest.loc[voyages_of_interest.discharge_event.notna(),'discharge_port_arrival_date'] = \
[ event['arrival_date'] for event in voy_df_for_iterations.discharge_event]

voyages_of_interest.loc[voyages_of_interest.discharge_event.notna(),'discharge_port_sailing_date'] = \
[ event['sailing_date'] for event in voy_df_for_iterations.discharge_event]

voyages_of_interest.loc[voyages_of_interest.discharge_event.notna(),'discharge_port_narrow_area'] = \
[ event['area_name_level0'] for event in voy_df_for_iterations.discharge_event]

voyages_of_interest.loc[voyages_of_interest.discharge_event.notna(),'discharge_port_wider_area'] = \
[ event['area_name_level2'] for event in voy_df_for_iterations.discharge_event]

voyages_of_interest.loc[voyages_of_interest.discharge_event.notna(),'discharge_port_area'] = \
[ event['area_name_level1'] for event in voy_df_for_iterations.discharge_event]

voyages_of_interest.loc[voyages_of_interest.discharge_event.notna(),'discharge_port_longitude'] = \
[ event['longitude'] for event in voy_df_for_iterations.discharge_event]

voyages_of_interest.loc[voyages_of_interest.discharge_event.notna(),'discharge_port_latitude'] = \
[ event['latitude'] for event in voy_df_for_iterations.discharge_event]

voyages_of_interest.loc[voyages_of_interest.discharge_event.notna(),'predicted_d'] = \
[ event['event_horizon'] for event in voy_df_for_iterations.discharge_event]

In [15]:
#validation that we do not have opening event only for the voyages that do not have prev voyage
voyages_of_interest.opening_event.isna().sum() == voyages_of_interest.prev_id.isna().sum()

True

In [16]:
def get_last_discharge_port(voyage):
    if not voyage['discharge_event']:
        return voyage['open_port']
    else:
        return next((e.port_name for e in reversed(voyage['events'][:-2] or []) if e.purpose=='Discharge'), None)
    
voyages_of_interest['last_discharge_port'] = voyages_of_interest.apply(get_last_discharge_port, axis=1)

In [17]:
def get_last_3_months_ind(laycan_from):
    if not pd.isnull(laycan_from):
        laycan_from = pd.to_datetime(laycan_from)
        return 1 if ((laycan_from.date()-date.today()).days<4*30) else 0
    else:
        return 0
    
voyages['last_3_months_ind'] = voyages['laycan_from'].apply(get_last_3_months_ind)

In [18]:
def get_sts_load_ind(load_event):
    return next((1 for d in load_event["event_details"] or [] if d.event_detail_type =='StS'), 0)

def get_sts_discharge_ind(discharge_event):
    return next((1 for d in discharge_event["event_details"] or [] if d.event_detail_type =='StS'), 0)


voyages_of_interest.loc[voyages_of_interest.discharge_event.notna(),'sts_discharge_ind'] = \
voyages_of_interest.loc[voyages_of_interest.discharge_event.notna(),'discharge_event'].apply(get_sts_discharge_ind)
voyages_of_interest.loc[voyages_of_interest.discharge_event.notna(),'sts_load_ind'] = \
voyages_of_interest.loc[voyages_of_interest.discharge_event.notna(),'load_event'].apply(get_sts_load_ind)

In [19]:
vessel_status_dict = {
    1:"Voyage", 2:"Breaking", 3:"Domestic Trade", 4:"FPSO", 5:"FPSO Conversion", 
    6:"Inactive", 7:"Storage Vessel", 9:"Conversion"
}
voyages_of_interest['vessel_status'] = voyages_of_interest.vessel_status_id.replace(vessel_status_dict)

In [20]:
unwanted_columns = ['vessel_class_id',
                    'vessel_status_id',
                    'commercial_operator_id',
                    'deleted',
                    'events',
                    'trade_id',
                    'vessel_status',
                    'start_date',
                    'end_date',
                    'charterer_id',
                    'ballast_bonus',
                    'ballast_bonus_type',
                    'cargo_type_id',
                    'cargo_group_id',
                    'cargo_type_source'
                   ]

voyages_of_interest.drop(unwanted_columns,axis=1,inplace=True)

In [21]:
voyages_of_interest.to_excel('voyages_data_likeV2.xlsx')